In [22]:
%load_ext autoreload
%autoreload 2
from simuq import QSystem
from simuq import Qubit
import numpy as np
from simuq.braket import BraketProvider
bp = BraketProvider()

def GenQS(n=12, p=3):
    qs = QSystem()
    ql = [Qubit(qs) for i in range(n)]
    link = [(i, (i + 1) % n) for i in range(n)]
    if p==3:    
        parameter_list = np.array(
        [
            0.5702193 * 2,
            -0.58631086,
            0.85160685 * 2,
            -1.7058538,
            0.29468536 * 2,
            -1.132814,
        ]
    )/2
    if p==2:
        parameter_list = np.array([-3.8034,1.2438, -1.2467,-2.4899])/2
    if p==1:
        parameter_list = np.array([-0.7854,-2.3562])/2

    for i in range(p):
       
        h = 0
        for q0, q1 in link:
            h += parameter_list[2 * i] * ql[q0].Z * ql[q1].Z
        qs.add_evolution(h, 1)
        h = 0
        for q0 in range(n):
            h += parameter_list[2 * i+1] * ql[q0].X
        qs.add_evolution(h, 1)
    return qs

def calc_cut(count):
    cut=0
    n_shots=sum(count.values())
    for key in count:
        cut+=count[key]*calc_weight(key)/n_shots
    return cut
def calc_weight(key):
    cut=0
    for i in range(len(key)):
        if key[i]!= key[(i+1)%len(key)]:
            cut+=1
    return cut

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
from simuq.braket import BraketIonQCircuit

state_prep = BraketIonQCircuit(12)
for i in range(12):
    state_prep.gpi2(i, np.pi / 2)

for p in [1, 2, 3]:
    qs = GenQS(12, p)
    bp.compile(
        qs,
        provider="ionq",
        device="Aria-1",
        aais="heisenberg",
        tol=0.01,
        trotter_num=4,
        state_prep=state_prep,
    )
    bp.run(shots=1024, on_simulator=True)
    result = bp.results()
    print(result)
    print(calc_cut(result))

Submitted.
{'000000110110': 0.0009765625, '000001101101': 0.0009765625, '000010001101': 0.0009765625, '000010010010': 0.0009765625, '000010110100': 0.0009765625, '000011010010': 0.0009765625, '000100100011': 0.0009765625, '000100101011': 0.0009765625, '000100110101': 0.0009765625, '000101001100': 0.0009765625, '000101010011': 0.0009765625, '000101010101': 0.001953125, '000101010111': 0.0009765625, '000101101001': 0.0009765625, '000110010011': 0.0009765625, '000110010101': 0.0009765625, '000110110010': 0.0009765625, '001001000101': 0.0009765625, '001001001001': 0.001953125, '001001010011': 0.001953125, '001001010101': 0.00390625, '001001011000': 0.0009765625, '001001011001': 0.0009765625, '001001011011': 0.0029296875, '001001101101': 0.0009765625, '001010001011': 0.0009765625, '001010010001': 0.0009765625, '001010010011': 0.0009765625, '001010010101': 0.00390625, '001010011001': 0.0029296875, '001010011011': 0.001953125, '001010100010': 0.0009765625, '001010100101': 0.0009765625, '00101